In [ ]:
#1 weather tool
from langchain.tools import tool
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from dotenv import load_dotenv
load_dotenv()
import os

class WeatherTool:
    os.environ['OPENWEATHERMAP_API_KEY'] = os.getenv("OPENWEATHERMAP_API_KEY")
    @tool
    def getWeather(city: str) ->str:
        """returns current weather for a city"""
        weather = OpenWeatherMapAPIWrapper()
        weather_data = weather.run(city)
        return weather_data

In [ ]:
#2
# #get lat long

import requests
from langchain.tools import tool
from dotenv import load_dotenv
load_dotenv()
import os
class LatLong:
    def __init__(self):
        self.url = 'https://api.geoapify.com/v1/geocode/search'
        self.api_key=os.getenv("GEOAPI_KEY")
    print("inside latlong")
    @tool
    def getLatitudeLongitude(self,city: str, country: str):
        """takes city and country as input paramter and returns dictionary with latitude and longitude of that input city in the input country"""
        print("inside func lat long")
        params = dict(
                        text=city,
                        countrycode=country,
                        apiKey=self.api_key
                    )
        resp = requests.get(url=self.url, params=params)
        data = resp.json()
        for feature in data.get("features", []):
            props = feature.get("properties", {})
            if props.get("result_type", "").lower() == "city":
                latitude = props.get("lat")
                longitude = props.get("lon")
                return{
                    "latitude": latitude,
                    "longitude": longitude

                }
        return {"latitude": None,"longitude": None}

In [ ]:
#3
#get attractions
import requests
from langchain.tools import tool
from dotenv import load_dotenv
from amadeus import Client, Location, ResponseError
load_dotenv()
import os
class TouristAttractions:
    def __init__(self):
        print("inside tourist attractions")
        self.apiKey=os.getenv("AMADEUS_API_KEY")
        self.apiSecret=os.getenv("AMADEUS_API_SECRET")
    @tool
    def getAttractions(self, longitude: float, latitude: float):
        """take latitude and longitude and returns tourist attractions"""
        print("inside func tourist attractions")
        try:
            print(self.apiKey)
            amadeus = Client(
                            client_id=self.apiKey,
                            client_secret=self.apiSecret
                        )
            #print(amadeus)
            #print(longitude)
            response = amadeus.shopping.activities.get(longitude=longitude,latitude=latitude,radius=1)  
            #print(response.data)
            return response.data
        except ResponseError as error:
            #print(error)
            return error

In [ ]:
#4 create tools array
from langchain.tools import Tool

geo = LatLong()
weatherTool = WeatherTool()
touristAttraction = TouristAttractions()

tools = [
    Tool(
        name="GetWeather",
        func=lambda input: weatherTool.getWeather(input.strip()),
        description="Get weather given a city (e.g., 'Paris')"
    ),
    Tool(
        name="GetLatLong",
        func=lambda input: geo.getLatitudeLongitude(*[i.strip() for i in input.split(",")]),
        description="Get latitude and longitude given 'city, countrycode' (e.g., 'Paris, FR')"
    ),
    Tool(
        name="GetTouristAttractions",
        func=lambda input: touristAttraction.getAttractions(*[float(i.strip()) for i in input.split(",")]),
        description="Get tourist attractions given 'longitude, latitude' (e.g., '2.3522, 48.8566')"
    )
]

In [ ]:
#5 use gemini model
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
response=llm.invoke("hi")
print(response)

In [ ]:
#6 bind model to tools
llm_with_tools=llm.bind_tools(tools) 

In [ ]:
#7 create graph
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph,MessagesState,START,END

In [ ]:
#8
builder=StateGraph(MessagesState)

In [ ]:
#9
SYSTEM_PROMPT="You are a travel agent tasked with getting me current weather, latitude longitude and tourist attractions for passed city. Only use Tools provided to you to find tourist attractions for passed input city and country and create travel itineary."

In [ ]:
#10
def travelAgent(state:MessagesState):
    user_question=state["messages"]
    input_question=[SYSTEM_PROMPT]+user_question
    response=llm_with_tools.invoke(input_question)
    return {
        "messages":[response]
    }

In [ ]:
#11
builder.add_node("llm_decision_step",travelAgent)

In [ ]:
#12
from langgraph.prebuilt import ToolNode
builder.add_node("tools",ToolNode(tools)) # to conver tools into langgraph tools use TollNode

In [ ]:
#13
builder.add_edge(START,"llm_decision_step")

In [ ]:
#14
from langgraph.prebuilt import tools_condition

builder.add_conditional_edges("llm_decision_step",
                        tools_condition)# for custom logic create router function else use this inbuilt function

In [ ]:
#15
builder.add_edge("tools","llm_decision_step")

In [ ]:
#16
react_graph=builder.compile()

In [ ]:
#17
from IPython.display import Image, display
display(Image(react_graph.get_graph().draw_mermaid_png()))

In [ ]:
#18
message=[HumanMessage(content="give me weather for Atlanta in US?")]

In [ ]:
#19
response=react_graph.invoke({"messages": message})

In [ ]:
#20
print(response)

In [ ]:
#21
response["messages"][-1].content

In [ ]:
#22
for m in response["messages"]:
    m.pretty_print()

In [ ]:
#23
message=[HumanMessage(content="give me tourist attractions for longitude: -3.69170868 and latitude: 40.41436995 ?")]

In [ ]:
#24
response=react_graph.invoke({"messages": message})

In [ ]:
for m in response["messages"]:
    m.pretty_print()